# Match 2018 NCSBE House Votes

Retrieve precinct-sorted 2018 general election results from [`2019-04-04 precinct_con_can_stats.zip`](https://dl.ncsbe.gov/index.html?prefix=ENRS/2018_11_06/precinct_sort/), match them to [parties from Ballotpedia 2018 races](https://docs.google.com/spreadsheets/d/1LmNoDfZH9lMtGh5kLwQG7Te7o3XugiarnK2zron72pE/edit#gid=0), and output [new 2018 North Carolina model data](https://docs.google.com/spreadsheets/d/1aMPAXJN7Km3fxglhHXPqsCl4TKGCE_JXooufT6KVom0/edit#gid=1429158271).

In [1]:
import os; os.environ['DYLD_LIBRARY_PATH'] = './.venv-NC/lib'
import pandas, editdistance, numpy, geopandas, re

## Import NCSBE Votes

In [2]:
df_2018 = pandas.read_csv('results_pct_20181106.txt.gz', sep='\t', dtype=str)
df_2018sldl = df_2018[df_2018['Contest Name'].str.startswith('NC HOUSE OF REPRESENTATIVES DISTRICT ')]

df_2016 = pandas.read_csv('precinct_sort_20161108.txt.gz', sep='\t', dtype=str)
df_2016uspres = df_2016[df_2016.contest_name == 'US PRESIDENT']

df_2018sldl

,County,Election Date,Precinct,Contest Group ID,Contest Type,Contest Name,Choice,Choice Party,Vote For,Election Day,One Stop,Absentee by Mail,Provisional,Total Votes,Real Precinct
956,BEAUFORT,11/06/2018,ABSENTEE,1082,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Jerry E. Langley,DEM,1,0,0,150,0,150,N
957,BEAUFORT,11/06/2018,SURBA,1082,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Jerry E. Langley,DEM,1,153,0,0,0,153,Y
958,BEAUFORT,11/06/2018,EDWAR,1082,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Jerry E. Langley,DEM,1,128,0,0,0,128,Y
959,BEAUFORT,11/06/2018,PINET,1082,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Jerry E. Langley,DEM,1,46,0,0,0,46,Y
960,BEAUFORT,11/06/2018,NCRK,1082,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Jerry E. Langley,DEM,1,81,0,0,0,81,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183413,WILKES,11/06/2018,101,1093,S,NC HOUSE OF REPRESENTATIVES DISTRICT 090,Sarah Stevens,REP,1,208,0,0,0,208,Y
183414,WILKES,11/06/2018,110,1093,S,NC HOUSE OF REPRESENTATIVES DISTRICT 090,Sarah Stevens,REP,1,98,0,0,0,98,Y
183415,WILKES,11/06/2018,ONE STOP PLEASANT HILL,1093,S,NC HOUSE OF REPRESENTATIVES DISTRICT 090,Sarah Stevens,REP,1,0,226,0,0,226,N
183416,WILKES,11/06/2018,125,1093,S,NC HOUSE OF REPRESENTATIVES DISTRICT 090,Sarah Stevens,REP,1,191,0,0,0,191,Y


## Import Ballotpedia Candidates

Match candidate names between Ballotpedia and NCSBE in each house district to later determine party votes.

In [3]:
df_candidates = pandas.read_csv('North Carolina State House Candidates - 2018 Candidates.csv')

def closest_name(name, names):
    if name in (numpy.nan, 'No candidate'):
        return None
    distances = sorted([(editdistance.distance(n, str(name)), n) for n in names])
    return distances[0][1]

DEMs, REPs, contests = list(), list(), list()

for (index, row) in df_candidates.iterrows():
    contest_name = 'NC HOUSE OF REPRESENTATIVES DISTRICT {:03d}'.format(int(row['State House District'][9:]))
    names = set(df_2018sldl[df_2018sldl['Contest Name'] == contest_name].Choice)
    DEMs.append(closest_name(row.Democrat, names))
    REPs.append(closest_name(row.Republican, names))
    contests.append(contest_name)

df_candidates['Democrat'], df_candidates['Republican'], df_candidates['Contest'] = DEMs, REPs, contests

df_candidates

,State House District,Winning Party,Incumbent Party,Democrat,Republican,Other,Contest
0,District 1,R,O,Ronald (Ron) Wesson,Edward C. Goodwin,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 001
1,District 2,R,R,Darryl D. Moss,Larry Yarborough,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 002
2,District 3,R,R,Barbara Lee,Michael Speciale,T. Lee Horne III (Libertarian Party),NC HOUSE OF REPRESENTATIVES DISTRICT 003
3,District 4,R,R,Da'Quan Marcell Love,Jimmy Dixon,Kevin E. Hayes (Constitution Party),NC HOUSE OF REPRESENTATIVES DISTRICT 004
4,District 5,D,D,Howard J. Hunter III,Phillip Smith,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 005
...,...,...,...,...,...,...,...
115,District 116,D,D,Brian Turner,Marilyn A. Brown,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 116
116,District 117,R,R,Gayle Kemp,Chuck McGrady,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 117
117,District 118,R,R,Rhonda Cole Schandevel,Michele D. Presnell,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 118
118,District 119,D,R,Joe Sam Queen,Mike Clampitt,NaN,NC HOUSE OF REPRESENTATIVES DISTRICT 119


## Count Precinct Party Votes

Create a new `df_2018new` DataFrame with vote counts for State representatives and U.S. President by party.

In [4]:
arrays = dict(county_desc=list(), precinct_code=list(), district=list(),
              incumbent=list(), winner=list(), sldl_votes_D=list(), sldl_votes_R=list(),
              uspres_votes_D=list(), uspres_votes_R=list())

groups1 = df_2018sldl.groupby(['County', 'Precinct', 'Contest Name']).groups

for (County, Precinct, Contest_Name) in groups1.keys():
    _df1 = df_2018sldl[df_2018sldl.County == County]
    _df2 = _df1[_df1.Precinct == Precinct]
    subdf_2018sldl = _df2[_df2['Contest Name'] == Contest_Name]
    
    arrays['county_desc'].append(County)
    arrays['precinct_code'].append(Precinct)
    
    row_candidates = df_candidates[df_candidates.Contest == Contest_Name].iloc[0]
    arrays['district'].append(row_candidates['State House District'][9:])
    arrays['incumbent'].append(row_candidates['Incumbent Party'])
    arrays['winner'].append(row_candidates['Winning Party'])
    
    try:
        DEM_name = row_candidates.Democrat
        DEM_row = subdf_2018sldl[subdf_2018sldl.Choice == DEM_name].iloc[0]
    except IndexError:
        arrays['sldl_votes_D'].append(0)
    else:
        arrays['sldl_votes_D'].append(int(DEM_row['Total Votes']))
            
    try:
        REP_name = row_candidates.Republican
        REP_row = subdf_2018sldl[subdf_2018sldl.Choice == REP_name].iloc[0]
    except IndexError:
        arrays['sldl_votes_R'].append(0)
    else:
        arrays['sldl_votes_R'].append(int(REP_row['Total Votes']))
    
    arrays['uspres_votes_D'].append(0)
    arrays['uspres_votes_R'].append(0)
    

groups2 = df_2018sldl.groupby(['County', 'Precinct']).groups

for (County, Precinct) in groups2.keys():
    _df1 = df_2016uspres[df_2016uspres.county_desc == County]
    subdf_2016uspres = _df1[_df1.precinct_code == Precinct]
    
    arrays['county_desc'].append(County)
    arrays['precinct_code'].append(Precinct)
    
    arrays['district'].append(None)
    arrays['incumbent'].append(None)
    arrays['winner'].append(None)
    
    arrays['sldl_votes_D'].append(0)
    arrays['sldl_votes_R'].append(0)
    
    try:
        DEM_row = subdf_2016uspres[subdf_2016uspres.candidate_name == 'Hillary Clinton'].iloc[0]
    except IndexError:
        arrays['uspres_votes_D'].append(0)
    else:
        arrays['uspres_votes_D'].append(int(DEM_row.votes))
            
    try:
        REP_row = subdf_2016uspres[subdf_2016uspres.candidate_name == 'Donald J. Trump'].iloc[0]
    except IndexError:
        arrays['uspres_votes_R'].append(0)
    else:
        arrays['uspres_votes_R'].append(int(REP_row.votes))
    

df_2018new = pandas.DataFrame(arrays)
print(df_2018new.shape)
print('SLDL Votes:', df_2018new.sldl_votes_D.sum(), df_2018new.sldl_votes_R.sum())
print('US Pres Votes:', df_2018new.uspres_votes_D.sum(), df_2018new.uspres_votes_R.sum())
df_2018new

(6828, 9)
SLDL Votes: 1853449 1786639
US Pres Votes: 2126510 2321481


,county_desc,precinct_code,district,incumbent,winner,sldl_votes_D,sldl_votes_R,uspres_votes_D,uspres_votes_R
0,ALAMANCE,01,64,R,R,341,1474,0,0
1,ALAMANCE,02,64,R,R,384,1589,0,0
2,ALAMANCE,035,64,R,R,828,1159,0,0
3,ALAMANCE,03C,63,R,R,557,941,0,0
4,ALAMANCE,03C,64,R,R,84,61,0,0
...,...,...,...,...,...,...,...,...,...
6823,YANCEY,07 BRU,None,None,None,0,0,91,163
6824,YANCEY,08 CRA,None,None,None,0,0,522,1260
6825,YANCEY,09 SOU,None,None,None,0,0,596,766
6826,YANCEY,10 PEN,None,None,None,0,0,106,270


## Assign Precinct Geography PSIDs

Read precinct PSID values from `geogs_2018.shp`, merge by county name and precinct ID.

In [5]:
geogs_2018 = geopandas.read_file('geogs_2018.shp', layer='geogs_2018')

geogs_2018.PSID = geogs_2018.PSID.str.replace(re.compile(r'^'), 'PSID:')

df_2018out = df_2018new.merge(geogs_2018, how='left',
                              left_on=('county_desc', 'precinct_code'),
                              right_on=('COUNTY_NAM', 'PREC_ID'))

df_2018out

,county_desc,precinct_code,district,incumbent,winner,sldl_votes_D,sldl_votes_R,uspres_votes_D,uspres_votes_R,PREC_ID,...,G18GHOR,G18LStHOR,G18LStSEN,G18LHOR,G18RStHOR,G18RStSEN,G18RHOR,G18UnaHOR,PSID,geometry
0,ALAMANCE,01,64,R,R,341,1474,0,0,01,...,0.0,0.0,0.0,0.0,1474.0,1462.0,1475.0,0.0,PSID:1158854937,POLYGON ((-79.43197308657012 35.88929191508174...
1,ALAMANCE,02,64,R,R,384,1589,0,0,02,...,0.0,0.0,0.0,0.0,1589.0,1572.0,1619.0,0.0,PSID:1158848961,POLYGON ((-79.53483899133799 36.04382100018476...
2,ALAMANCE,035,64,R,R,828,1159,0,0,035,...,0.0,0.0,0.0,0.0,1159.0,1167.0,1185.0,0.0,PSID:1158850573,POLYGON ((-79.50051299120325 36.15426500013934...
3,ALAMANCE,03C,63,R,R,557,941,0,0,03C,...,0.0,0.0,0.0,0.0,1002.0,1012.0,1034.0,0.0,PSID:1158854377,POLYGON ((-79.52032399117273 36.08219700003401...
4,ALAMANCE,03C,64,R,R,84,61,0,0,03C,...,0.0,0.0,0.0,0.0,1002.0,1012.0,1034.0,0.0,PSID:1158854377,POLYGON ((-79.52032399117273 36.08219700003401...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6823,YANCEY,07 BRU,None,None,None,0,0,91,163,07 BRU,...,0.0,0.0,0.0,4.0,143.0,135.0,145.0,0.0,PSID:1158854169,POLYGON ((-82.19135299166359 35.97501800047963...
6824,YANCEY,08 CRA,None,None,None,0,0,522,1260,08 CRA,...,0.0,0.0,0.0,26.0,1096.0,997.0,1131.0,0.0,PSID:1158852683,POLYGON ((-82.18617999165116 35.94352800006673...
6825,YANCEY,09 SOU,None,None,None,0,0,596,766,09 SOU,...,0.0,0.0,0.0,21.0,626.0,604.0,633.0,0.0,PSID:1158854577,POLYGON ((-82.22187599103364 35.86516199983036...
6826,YANCEY,10 PEN,None,None,None,0,0,106,270,10 PEN,...,0.0,0.0,0.0,7.0,225.0,221.0,232.0,0.0,PSID:1158853255,POLYGON ((-82.26803599172993 35.76373800026749...


## Group Votes by Precinct

Add up all votes, grouping by precinct and concatenating multiple districts and incumbents.

In [6]:
grouped = df_2018out.groupby(['county_desc', 'precinct_code'])

def semicolon(series):
    return ';'.join({str(v) for v in series.values if v and v is not numpy.nan})

def doit(df):
    #print(df)
    return pandas.DataFrame({
        'psid': [semicolon(df.PSID)],
        'district': [semicolon(df.district)],
        'winner': [semicolon(df.winner)],
        'incumbent': [semicolon(df.incumbent)],
        'sldl_votes_D': [df.sldl_votes_D.sum()],
        'sldl_votes_R': [df.sldl_votes_R.sum()],
        'uspres_votes_D': [df.uspres_votes_D.sum()],
        'uspres_votes_R': [df.uspres_votes_R.sum()],
    })

df_2018final = grouped.apply(doit)

df_2018final

psid district winner incumbent  \
county_desc precinct_code                                                
ALAMANCE    01            0  PSID:1158854937       64      R         R   
            02            0  PSID:1158848961       64      R         R   
            035           0  PSID:1158850573       64      R         R   
            03C           0  PSID:1158854377    63;64      R         R   
            03N           0  PSID:1158852503       64      R         R   
...                                      ...      ...    ...       ...   
YANCEY      07 BRU        0  PSID:1158854169      118      R         R   
            08 CRA        0  PSID:1158852683      118      R         R   
            09 SOU        0  PSID:1158854577      118      R         R   
            10 PEN        0  PSID:1158853255      118      R         R   
            11 PRI        0  PSID:1158852881      118      R         R   

                             sldl_votes_D  sldl_votes_R  uspres_votes_D  \
county_desc precinct_code                                                 
ALAMANCE    01            0           341          1474             411   
            02            0           384          1589             403   
            035           0           828          1159             995   
            03C           0           641          1002             679   
            03N           0           745           964            1036   
...                                   ...           ...             ...   
YANCEY      07 BRU        0           103           143              91   
            08 CRA        0           580          1096             522   
            09 SOU        0           620           626             596   
            10 PEN        0           122           225             106   
            11 PRI        0           255           557             198   

                             uspres_votes_R  
county_desc precinct_code                    
ALAMANCE    01            0            1865  
            02            0            2004  
            035           0            1485  
            03C           0            1059  
            03N           0            1160  
...                                     ...  
YANCEY      07 BRU        0             163  
            08 CRA        0            1260  
            09 SOU        0             766  
            10 PEN        0             270  
            11 PRI        0             639  

[3052 rows x 8 columns]

In [7]:
df_2018final.to_csv('df_2018out.csv')